## OOI NetCDF Data Access Part 1

You will learn:

* how to navigate the OOI dataset
* how to request data through the OOI M2M API
* how to find and navigate your personal THREDDS server directory

### Navigating the OOI Dataset
In this example we show how to programatically download and work with OOI NetCDF time series data. We will use data from the Global Argentine Basin Flanking Subsurface Mooring A Mooring Riser CTD at 30 m. 

You will need the Reference Designator, Stream and Data Delivery Method to make the request to the M2M API, as seen in the screenshot highlighted below. These, along with additional useful information about the instrument, can be found here: http://ooi.visualocean.net/instruments/view/GA03FLMA-RIM01-02-CTDMOG040

![GA03FLMA-RIM01-02-CTDMOG040](images/GA03FLMA-RIM01-02-CTDMOG040.png)

More information:

[Instrument Pages](https://oceanobservatories.org/instruments/) Descriptions and documentation about OOI instrumentation  
[How to decipher a Reference Designator](https://oceanobservatories.org/knowledgebase/how-to-decipher-a-reference-designator)  Knowledge base entry further breaking down Reference Designators  
[Understanding Data Streams and Parameters](https://oceanobservatories.org/knowledgebase/understanding-data-streams-and-parameters/)  Knowledge base entry on Streams and Parameters.

### Requesting data through the OOI M2M API

A typical M2M data request URL looks like this:

https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/GA03FLMA/RIM01/02-CTDMOG040/recovered_inst/ctdmo_ghqr_instrument_recovered?beginDT=2015-03-17T01:01:01.500Z&endDT=2018-09-01T01:01:01.500Z

Let's break it down:

* Base URL: https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv
* Reference Designator: GA03FLMA/RIM01/02-CTDMOG040
* Data Delivery Method: recovered_inst
* Stream Name: ctdmo_ghqr_instrument_recovered
* Time Range: beginDT=2015-03-17T01:01:01.500Z&endDT=2018-09-01T01:01:01.500Z



In [1]:
import requests
import time

Login in at https://ooinet.oceanobservatories.org/ and obtain your <b>API username</b> and <b>API token</b> under your profile (top right corner) or use credentials provided below for now. 

If you use your personal login, you will generate a personal THREDDS directory hosting only the data you request.

In [2]:
username = 'OOIAPI-D8S960UXPK4K03'
token = 'IXL48EQ2XY'

Specify your inputs.

In [3]:
refdes = 'GA03FLMA-RIM01-02-CTDMOG040'
method = 'recovered_inst'
stream = 'ctdmo_ghqr_instrument_recovered'
beginDT = '2015-03-17T01:01:01.500Z' # this time range encompasses all three deployments of this instrument.
endDT = '2018-09-01T01:01:01.500Z'

Build the GET request URL and send the request to the M2M API endpoint.

In [4]:
base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv'

data_request_url ='/'.join((base_url,refdes[:8],refdes[9:14],refdes[15:],method,stream))
params = {
    'beginDT':beginDT,
    'endDT':endDT,   
}
r = requests.get(data_request_url, params=params, auth=(username, token))
data = r.json()

Examine the response.

In [5]:
import json
print(json.dumps(data, indent=2))

{
  "requestUUID": "043f57c6-eee7-40b6-ad7e-06fc498302e8",
  "outputURL": "https://opendap.oceanobservatories.org/thredds/catalog/ooi/ooidatateam@gmail.com/20180815T154831-GA03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/catalog.html",
  "allURLs": [
    "https://opendap.oceanobservatories.org/thredds/catalog/ooi/ooidatateam@gmail.com/20180815T154831-GA03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/catalog.html",
    "https://opendap.oceanobservatories.org/async_results/ooidatateam@gmail.com/20180815T154831-GA03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered"
  ],
  "sizeCalculation": 6207148,
  "timeCalculation": 60,
  "numberOfSubJobs": 38
}


Notice that the timeCalculation key is attempting to give you a rough estimate of how long it will take to fulfill your data request.

### Navigating your THREDDS directory.

Check if the request has completed.

In [6]:
%%time
check_complete = data['allURLs'][1] + '/status.txt'
for i in range(1800): 
    r = requests.get(check_complete)
    if r.status_code == requests.codes.ok:
        print('request completed')
        break
    else:
        time.sleep(1)

request completed
CPU times: user 944 ms, sys: 66 ms, total: 1.01 s
Wall time: 1min


Check out the THREDDS directory.

In [7]:
data['allURLs'][0]

'https://opendap.oceanobservatories.org/thredds/catalog/ooi/ooidatateam@gmail.com/20180815T154831-GA03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/catalog.html'

See all datasets on the THREDDS server under your username.  
https://opendap.oceanobservatories.org/thredds/catalog/ooi/ooidatateam@gmail.com/catalog.html

Check out the Apache directory.

In [8]:
data['allURLs'][1]

'https://opendap.oceanobservatories.org/async_results/ooidatateam@gmail.com/20180815T154831-GA03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered'

See all datasets at the Apache server under your username.  
https://opendap.oceanobservatories.org/async_results/ooidatateam@gmail.com

#### Important
The data you have requested will persist at these locations for the next 6 months. If you have the dataset you are looking for, you do not need to request these data again and again.

Now that the data is ready and waiting to be used, copy the THREDDS url to your clip board and proceed to Part 2.